In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('/projects/cft_vlm/finetune')
import transformers
from transformers import AutoProcessor
import datasets
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", use_fast=True)


You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [1]:
from transformers import Qwen2_5_VLForConditionalGeneration

og_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
  "Qwen/Qwen2.5-VL-3B-Instruct",
  torch_dtype="auto",
  device_map="auto", 
  attn_implementation="flash_attention_2")

checkpoint_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
  "/projects/cft_vlm/.checkpoint/Qwen/Qwen2.5-VL-3B-Instruct-openbiomedvid/checkpoint-100",
  torch_dtype="auto",
  device_map="auto",
  attn_implementation="flash_attention_2")

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
og_language_model = og_model.language_model
c_language_model = checkpoint_model.language_model

# Compare the two models
for name, og_param in og_language_model.named_parameters():
    c_param = c_language_model.get_parameter(name)
    if not og_param.equal(c_param):
        print(f"Parameter {name} differs between the original and checkpoint model.")
    else:
        print(f"Parameter {name} is the same in both models.")

Parameter embed_tokens.weight differs between the original and checkpoint model.
Parameter layers.0.self_attn.q_proj.weight differs between the original and checkpoint model.
Parameter layers.0.self_attn.q_proj.bias differs between the original and checkpoint model.
Parameter layers.0.self_attn.k_proj.weight differs between the original and checkpoint model.
Parameter layers.0.self_attn.k_proj.bias is the same in both models.
Parameter layers.0.self_attn.v_proj.weight differs between the original and checkpoint model.
Parameter layers.0.self_attn.v_proj.bias differs between the original and checkpoint model.
Parameter layers.0.self_attn.o_proj.weight differs between the original and checkpoint model.
Parameter layers.0.mlp.gate_proj.weight differs between the original and checkpoint model.
Parameter layers.0.mlp.up_proj.weight differs between the original and checkpoint model.
Parameter layers.0.mlp.down_proj.weight differs between the original and checkpoint model.
Parameter layers.0.

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:2 and cuda:1! (when checking argument for argument other in method wrapper_CUDA__equal)

In [7]:
for name, mat in og_model.language_model.named_parameters():
  print(name)

embed_tokens.weight
layers.0.self_attn.q_proj.weight
layers.0.self_attn.q_proj.bias
layers.0.self_attn.k_proj.weight
layers.0.self_attn.k_proj.bias
layers.0.self_attn.v_proj.weight
layers.0.self_attn.v_proj.bias
layers.0.self_attn.o_proj.weight
layers.0.mlp.gate_proj.weight
layers.0.mlp.up_proj.weight
layers.0.mlp.down_proj.weight
layers.0.input_layernorm.weight
layers.0.post_attention_layernorm.weight
layers.1.self_attn.q_proj.weight
layers.1.self_attn.q_proj.bias
layers.1.self_attn.k_proj.weight
layers.1.self_attn.k_proj.bias
layers.1.self_attn.v_proj.weight
layers.1.self_attn.v_proj.bias
layers.1.self_attn.o_proj.weight
layers.1.mlp.gate_proj.weight
layers.1.mlp.up_proj.weight
layers.1.mlp.down_proj.weight
layers.1.input_layernorm.weight
layers.1.post_attention_layernorm.weight
layers.2.self_attn.q_proj.weight
layers.2.self_attn.q_proj.bias
layers.2.self_attn.k_proj.weight
layers.2.self_attn.k_proj.bias
layers.2.self_attn.v_proj.weight
layers.2.self_attn.v_proj.bias
layers.2.self_at

In [7]:
model.enable_input_require_grads()

In [3]:
import torch

from qwenvl.argument import ModelArguments, DataArguments, ProcessingArguments
torch.set_num_threads(1)
parser = transformers.HfArgumentParser((
    ModelArguments,
    DataArguments,
    ProcessingArguments,
))

from qwenvl.data import logger
from qwenvl.train import set_processor, make_data_module

model_args, data_args, proc_args = ModelArguments(), DataArguments(), ProcessingArguments()
logger.info("Counting tokens, not training.")
processor = transformers.AutoProcessor.from_pretrained(
    model_args.model_name_or_path,
    use_fast=True,
)
data_args.dataset_use = 'openbiomedvid'
processor = set_processor(processor, proc_args, data_args)

2025-06-10 19:56:59,834 - qwenvl.data - INFO - Counting tokens, not training.


In [4]:
data_module = make_data_module(
    processor=processor, data_args=data_args, proc_args=proc_args, for_training=True
)

2025-06-10 19:57:02,066 - qwenvl.data - INFO - Loading dataset connectthapa84/OpenBiomedVid from /projects/cft_vlm/datasets/openbiomedvid/data/dataset


2025-06-10 19:57:02,087 - qwenvl.data - INFO - bin_pkl_path.exists()=True, self.need_num_content_tokens()=False
2025-06-10 19:57:02,089 - qwenvl.data - INFO - Loading bins from pickle file /projects/cft_vlm/datasets/openbiomedvid/data/dataset/bins.pkl.


In [5]:
ds = data_module['train_dataset']
ds.collate_fn(ds[0])

TypeError: unsupported operand type(s) for /: 'int' and 'list'

In [26]:
from pathlib import Path
import torch
from qwenvl.new.predict import get_trainer_args, get_generated_text, save_result
from qwenvl.new.train import rank0_make_data_module
from transformers import Seq2SeqTrainer

data_args, proc_args, eval_args = DataArguments(), ProcessingArguments(), EvalArguments()

data_args.dataset_use = 'vqa-rad'
data_args.split = 'test'

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    eval_args.model_name_or_path,
    torch_dtype=torch.float16,
    attn_implementation="flash_attention_2"
)
processor = AutoProcessor.from_pretrained(eval_args.model_name_or_path)
processor = set_processor(processor, proc_args, data_args)
data_module = rank0_make_data_module(
    processor=processor,
    data_args=data_args,
    proc_args=proc_args,
    for_training=False
)
trainer_args = get_trainer_args(eval_args)

trainer = Seq2SeqTrainer(
    model=model,
    processing_class=processor,
    args=trainer_args,
    **data_module
)
output = trainer.predict(trainer.eval_dataset)
generated_text = get_generated_text(output, processor.tokenizer)
if eval_args.output_dir is not None:
  output_dir = Path(eval_args.output_dir)
else:
  output_dir = Path(eval_args.model_name_or_path)
save_result(data_module['eval_dataset'], generated_text, output_dir)
    

ModuleNotFoundError: No module named 'mpi4py'

In [ ]:
from qwenvl.data import OpenpmcDataset


ds = OpenpmcDataset(
    processor=processor,
    data_args=data_args,
    proc_args=proc_args,
    split='train',
)


In [4]:
import numpy as np
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    torch_dtype="auto",
    device_map="auto", 
    attn_implementation="flash_attention_2"
)
from PIL import Image as PILImage

processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", use_fast=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/fs01/projects/cft_vlm/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:2479: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [22]:
img1 = PILImage.fromarray((np.ones((224, 224, 3)) * np.array([0, 0, 255])).astype(np.uint8))
img2 = PILImage.fromarray((np.ones((224, 224, 3)) * np.array([0, 255, 0])).astype(np.uint8))
conversations = [
  [
    {
      "role": "user",
      "content": [
        {
          "type": 'image',
          "image": img1,
        },
        {
          "type": 'text',
          "text": "What is the color of this image?"
        }
      ]
    },
  ],
  [
    {
      "role": "user",
      "content": [
        {
          "type": 'image',
          "image": img2,
        },
        {
          "type": 'text',
          "text": "What is the color of this image?"
        }
      ]
    }
  ]
]
texts = processor.tokenizer.apply_chat_template(
    conversations[0],
    tokenize=False,
    add_generation_prompt=True,
)
inputs = processor(
    images=[[img1], [img2]][0],
    text=texts,
    return_tensors="pt",
    padding_side='right',
)
output = model.generate(**inputs, max_new_tokens=10)

In [23]:
processor.tokenizer.batch_decode(output, skip_special_tokens=True)

['system\nYou are a helpful assistant.\nuser\nWhat is the color of this image?\nassistant\nThe image you provided is a solid blue color.']